In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from time import time
from sklearn.preprocessing import QuantileTransformer
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from time import time
start = time()
train = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv')
ss = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv')
print(f'Took {time() - start :.2f} seconds.')

In [ ]:
train.to_feather('train')
test.to_feather('test')
ss.to_feather('sample_solution')

In [ ]:
#Count number of nans per row
train['nan_count'] = np.sum(train.isnull(), axis=1)
test['nan_count'] = np.sum(test.isnull(), axis=1)

#Recording which values were null
null_msk_train =train.isnull().values
null_msk_test = test.isnull().values

In [ ]:
##########################################
##########################################
#Median imputing the nans
##########################################
##########################################

#Count number of nans per row
train['nan_count'] = np.sum(train.isnull(), axis=1)
test['nan_count'] = np.sum(test.isnull(), axis=1)

#Recording which values were null
null_msk_train =train.isnull().values
null_msk_test = test.isnull().values


#Making Nans less than the lowest value in each col by 1
def median_impute(df, FEATURES):
    for feat in FEATURES:
        df.loc[df[feat].isnull(), feat] = np.nanmedian(df[feat].values) - 1
FEATURES = [feat for feat in train.columns if 'f' in feat]
        
start = time()
median_impute(train, FEATURES)
median_impute(test, FEATURES)

print(f'Took {time() - start :.2f} seconds')

#Memory downsize
from tqdm.notebook import tqdm
for feat in tqdm(FEATURES):
    train[feat] = train[feat].astype(np.float32)
    test[feat] = test[feat].astype(np.float32)
    
#Recording the min df to feather
train.drop('nan_count', axis=1).to_feather('train_median')
test.drop('nan_count', axis=1).to_feather('test_median')

#Rank Gauss
start = time()
qt = QuantileTransformer(n_quantiles=1000, output_distribution='normal')
train[FEATURES] = qt.fit_transform(train[FEATURES])
test[FEATURES] = qt.transform(test[FEATURES])
print(f'Took {time() - start :.2f} seconds')

#Memory downsize
from tqdm.notebook import tqdm
for feat in tqdm(FEATURES):
    train[feat] = train[feat].astype(np.float32)
    test[feat] = test[feat].astype(np.float32)
    
#Saving median Rank Gaussed 
train.drop('nan_count', axis=1).to_feather('train_rg')
test.drop('nan_count', axis=1).to_feather('test_rg')

#Replacing median values with -6
train = train * ~null_msk_train  + -6 * null_msk_train
test = test * ~null_msk_test  + -6 * null_msk_test

#Saving min Rank Gaussed with nan_count column 
train.to_feather('train_rg_min')
test.to_feather('test_rg_min')